Pranjal Kashyap

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("Downloads/Bengaluru_House_Data.csv")
data

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
for column in data.columns:
    print(data[column].value_counts())
    print('*'*20)

In [ ]:
data['location'].fillna(data['location'].mode()[0], inplace = True)

In [ ]:
data['size'].fillna(data['size'].mode()[0], inplace = True)

In [ ]:
data['society'].fillna(data['society'].mode()[0], inplace = True)

In [ ]:
data['bath'].fillna(data['bath'].mean(), inplace = True)

In [ ]:
data['balcony'].fillna(data['balcony'].mode()[0], inplace = True)

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
data.drop(columns = ['area_type','availability'], inplace=True)

In [ ]:
data

In [ ]:
data.describe()

In [ ]:
data['bhk'] = data['size'].str.split().str.get(0).astype(int)

In [ ]:
data[data.bhk > 20]

In [ ]:
data['total_sqft'].unique()

In [ ]:
def convertRange(x):

    temp = x.split('-')
    if len(temp) == 2:
        return (float(temp[0]) + float(temp[1]))/2
    try:
        return float(x)
    except:
        return None
        

In [ ]:
data['total_sqft'] = data['total_sqft'].apply(convertRange)

In [ ]:
data['price_per_sqft'] = data['price']*100000 / data['total_sqft']

In [ ]:
data['price_per_sqft']

In [ ]:
data.describe()

In [ ]:
data['location'].value_counts()

In [ ]:
data['location'] = data['location'].apply(lambda x: x.strip())
location_count = data['location'].value_counts()
location_count

In [ ]:
loctaion_count_less10 = location_count[location_count <= 10]
loctaion_count_less10

In [ ]:
data['location'] = data['location'].apply(lambda x: 'other' if x in loctaion_count_less10 else x)

In [ ]:
data['location'].value_counts()

Outlier

In [ ]:
data.describe()

In [ ]:
(data['total_sqft']/data['bhk']).describe()

In [ ]:
data = data[(data['total_sqft']/data['bhk']) >= 300]
data.describe()

In [ ]:
data.shape

In [ ]:
data['price_per_sqft'].describe()

In [ ]:
def remove_outliers_sqft(df):
    df_output = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)

        st = np.std(subdf.price_per_sqft)

        gen_df = subdf[(subdf.price_per_sqft > (m-st)) & (subdf.price_per_sqft <= (m+st))]
        df_output = pd.concat([df_output,gen_df], ignore_index=True)
    return df_output
data = remove_outliers_sqft(data)
data.describe()

In [ ]:
def bh_outlier_remover(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]

            }

        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices, axis= 'index')

In [ ]:
data = bh_outlier_remover(data)

In [ ]:
data.shape

In [ ]:
data

 Cleaned Data

In [ ]:
data.head()

In [ ]:
data.to_csv("Cleaned_Data.csv")

In [ ]:
x = data.drop(columns= ['price'])
y = data['price']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
column_trans = make_column_transformer((OneHotEncoder(sparse_output=False), ['location']), remainder='passthrough')

In [ ]:
ss = StandardScaler() 

In [ ]:
lr = LinearRegression()

In [ ]:
pipe = make_pipeline(
    OneHotEncoder(handle_unknown='ignore'),
    StandardScaler(with_mean=False),
    LinearRegression())

In [ ]:
pipe.fit(x_train,y_train)

In [ ]:
pred = pipe.predict(x_test)

In [ ]:
r2_score(y_test, pred)*100

In [ ]:
from sklearn.linear_model import Lasso,Ridge

In [ ]:
la = Lasso()

In [ ]:
pipe = make_pipeline(column_trans, ss, la)

In [ ]:
pipe.fit(x_train, y_train)

In [ ]:
pred1 = pipe.predict(x_test)
r2_score(y_test, pred1)*100

In [ ]:
ri = Ridge()

In [ ]:
pipe = make_pipeline(column_trans, ss, ri)

In [ ]:
pipe.fit(x_train, y_train)

In [ ]:
pred2 = pipe.predict(x_test)
r2_score(y_test, pred2)*100

In [ ]:
import pickle

In [ ]:
pickle.dump(pipe, open('RidgeModel.pkl','wb'))